In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import  Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

In [4]:
from keras.datasets import cifar10

In [5]:
# Inputs:  x_train, x_test 
# Outputs: y_train, y_test

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [ ]:
# print(x_test)

In [6]:
## Normalization of preprocessed data

x_train = x_train.astype('float32')
x_test  = x_test.astype('float32')

 # To make all data b/w 0 and 1 form

x_train = x_train / 255.0
x_test  = x_test / 255.0

In [7]:
# Normalizing the output using the one-hot encoding for categorical data
y_train = np_utils.to_categorical(y_train)
y_test  = np_utils.to_categorical(y_test)

# Calculating the number of classes(How my data is group by classes) in Neural Network: 
# we use this to see how many neurons we need to compress the final layer
class_num = y_test.shape[1] # Called Shape to see all the classes for the data

In [8]:
model = Sequential()

In [9]:
model.add(Conv2D(32, (3,3), input_shape = x_train.shape[1:], padding = 'same'))
model.add(Activation('relu'))

In [10]:
model.add(Conv2D(32, (3,3), input_shape = (3, 32, 32), activation='relu', padding = 'same') )

In [11]:
# Dropout function to prevent overfitting
model.add(Dropout(0.2))

In [12]:
# To normalize the input to the next layer: We ensure that the network always create a activation with the same distribution
# Keep normaliziling the data at each layer
model.add(BatchNormalization())

In [13]:
model.add(Conv2D(64, (3,3), padding = 'same', activation='relu'))
model.add(Activation('relu'))

In [14]:
# To abstract the unnecessary part of the image
# and keeping only the part of the image that the model thinks that are relevant
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [15]:
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(256, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [16]:
model.add(Flatten())
model.add(Dropout(0.2))

In [17]:
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [18]:
 model.add(Dense(class_num))
 model.add(Activation('softmax'))

In [19]:
epochs = 50

optimizer = 'adam'

In [20]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [21]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 dropout (Dropout)           (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 64)        18496     
                                                        

In [22]:
seed = 21
np.random.seed(seed)
model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=epochs, batch_size=64)

Epoch 1/50
782/782 [==============================] - 39s 48ms/step - loss: 1.5288 - accuracy: 0.4563 - val_loss: 1.3975 - val_accuracy: 0.5308
Epoch 2/50
782/782 [==============================] - 37s 47ms/step - loss: 1.0011 - accuracy: 0.6449 - val_loss: 0.8979 - val_accuracy: 0.6871
Epoch 3/50
782/782 [==============================] - 37s 47ms/step - loss: 0.8150 - accuracy: 0.7177 - val_loss: 0.7642 - val_accuracy: 0.7324
Epoch 4/50
782/782 [==============================] - 37s 47ms/step - loss: 0.7226 - accuracy: 0.7486 - val_loss: 0.6836 - val_accuracy: 0.7594
Epoch 5/50
782/782 [==============================] - 37s 47ms/step - loss: 0.6564 - accuracy: 0.7710 - val_loss: 0.6279 - val_accuracy: 0.7809
Epoch 6/50
782/782 [==============================] - 37s 48ms/step - loss: 0.6152 - accuracy: 0.7849 - val_loss: 0.6095 - val_accuracy: 0.7870
Epoch 7/50
782/782 [==============================] - 37s 47ms/step - loss: 0.5741 - accuracy: 0.8018 - val_loss: 0.5809 - val_accuracy:

In [23]:
scores = model.evaluate(x_test, y_test, verbose = 0)
print(scores[1]*100)

86.04000210762024
